In [1]:
from qat.lang.AQASM import Program, H, CNOT

# Create a Program
qprog = Program()
# Number of qbits
nbqbits = 2
# Allocate some qbits
qbits = qprog.qalloc(nbqbits)

# Apply some quantum Gates
H(qbits[0])
CNOT(qbits[0], qbits[1])

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# Import a Quantum Processor Unit Factory (the default one)
from qat.qpus import get_default_qpu

# Create a Quantum Processor Unit
qpu = get_default_qpu()

# Create a job
job = circuit.to_job()

# Submit the job to the QPU
result = qpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("State %s amplitude %s" % (sample.state, sample.amplitude))

ImportError: dlopen(/Users/rajarshi/miniforge3/envs/myqlm/lib/python3.9/site-packages/qat/__init__.so, 0x0002): Library not loaded: @rpath/lib/libpython3.9.dylib
  Referenced from: <E8C3CBC8-276E-3534-8DFE-16C5457CA465> /Users/rajarshi/miniforge3/envs/myqlm/lib/python3.9/site-packages/qat/__init__.so
  Reason: tried: '/Library/Frameworks/Python.framework/Versions/3.9/lib/libpython3.9.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.9/lib/libpython3.9.dylib' (no such file), '/opt/homebrew/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/libpython3.9.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/libpython3.9.dylib' (no such file), '/usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/libpython3.9.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/python@3.9/Frameworks/Python.framework/Versio

In [3]:
qbits?

Type:        QRegister
String form: QReg(q[0]..q[1])
Length:      2
File:        ~/miniconda3/envs/qc/lib/python3.10/site-packages/qat/lang/AQASM/bits.so
Docstring:  
Class for registers of qbits. They should only be
declared through the qalloc method of the Program class.
Qbits inside a register can be accessed
similarly to elements of an array (i.e using brackets and slices).
